<a href="https://colab.research.google.com/github/MishNia/GenNer/blob/main/entity_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate labels on the sampled pile corpus with ~50k passages

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Dirty code to deal with "Field larger than field limit" error

In [ ]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [ ]:
# Define your text passages
datapath = 'content/sampled_dataset_tokenised.csv'
df = pd.read_csv(datapath, engine='python', on_bad_lines='skip')

text_passages = df['original_text'].tolist()

yay. read the data into a dataframe.

blocks:

https://www.shanelynn.ie/pandas-csv-error-error-tokenizing-data-c-error-eof-inside-string-starting-at-line/

https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072

https://stackoverflow.com/questions/52105659/pandas-read-csv-unexpected-end-of-data-error


In [ ]:
df.head()

,Unnamed: 0,original_text,tokenized_text,pile_set_name
0,0,"For Father’s Day this year, why not give him (...","[101, 1370, 4505, 787, 188, 2295, 1142, 1214, ...",Pile-CC
1,1,The Senate has joined the Federal Trade Commis...,"[101, 1109, 3279, 1144, 1688, 1103, 3467, 5820...",Pile-CC
2,2,"This weekend during the TCA event, it was anno...","[101, 1188, 5138, 1219, 1103, 157, 11356, 1856...",Pile-CC
3,3,A Jeweler's Eye for Flaw\n\nStories\n\nChristi...,"[101, 138, 23624, 1200, 112, 188, 9329, 1111, ...",Pile-CC
4,4,SHARE\n\nMcDonald’s Family Classic turns out t...,"[101, 17730, 12426, 2036, 9092, 787, 188, 3921...",Pile-CC


In [ ]:
df['original_text'][0]

'For Father’s Day this year, why not give him (or yourself) a gift he would really want? Cigar Art Prints and Canvas Wraps are available for limited time only at 30% off normal prices! A huge variety of shots are available, everyone is sure to find something they will love! … Continue reading →\n\nI have an extra copies of Portal, Half-Life 2 and Half Life 2 Ep 1 to gift through Steam. These are some of my favorite games of all time and I would love to share them with viewers 🙂 Leave a comment below on the site with which one you … Continue reading →'

In [ ]:
len(text_passages)

63477

In [ ]:
from openai import OpenAI
import time

# Set your OpenAI API key
client = OpenAI(
     api_key= '<key>'
)

# Function to call OpenAI API for named entity recognition
def analyze_text(text):
    messages = [{
        "role": "system",
        "content": "Given a passage, your task is to extract all entities and identify their entity types. The output should be in a list of tuples of the following format: [('entity 1', 'type of entity 1'), ... ]."
    }, {
        "role": "user",
        "content": text
    }]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        max_tokens=256,  # Adjust as needed
        stop=["\n", "<|endoftext|>"],
        temperature=0,
        n=1
    )
    return response

# Main function to process text passages
def process_text_passages(passage):
    response = analyze_text(passage)
    print(response)
    entities = response.choices[0].message.content
    print(f"\n Entities are {entities}")
    return entities

In [ ]:
import time

counter = 0
for index, row in df.iterrows():
    if counter % 5 == 0 and counter != 0:  # After every 5 calls, sleep for 1 second
        time.sleep(1)
    df.at[index, 'generated_entities'] = process_text_passages(row['original_text'])
    counter += 1

    if counter % 10 == 0:
      print("Processed 10 passages")
      df.head()
      break

ChatCompletion(id='chatcmpl-9AeUAWOCEVuzGOCmUDKNiOhXiTjLk', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="[('Father’s Day', 'event'), ('Cigar Art Prints', 'product'), ('Canvas Wraps', 'product'), ('Portal', 'product'), ('Half-Life 2', 'product'), ('Half Life 2 Ep 1', 'product'), ('Steam', 'platform')]", role='assistant', function_call=None, tool_calls=None))], created=1712325934, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_b28b39ffa8', usage=CompletionUsage(completion_tokens=59, prompt_tokens=182, total_tokens=241))

 Entities are [('Father’s Day', 'event'), ('Cigar Art Prints', 'product'), ('Canvas Wraps', 'product'), ('Portal', 'product'), ('Half-Life 2', 'product'), ('Half Life 2 Ep 1', 'product'), ('Steam', 'platform')]
ChatCompletion(id='chatcmpl-9AeUCLIoNg3S9vEiSiX42GBhirnMs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="[('Senate', 'Gove

In [ ]:
df.head()

,Unnamed: 0,original_text,tokenized_text,pile_set_name,generated_entities
0,0,"For Father’s Day this year, why not give him (...","[101, 1370, 4505, 787, 188, 2295, 1142, 1214, ...",Pile-CC,"[('Father’s Day', 'event'), ('Cigar Art Prints..."
1,1,The Senate has joined the Federal Trade Commis...,"[101, 1109, 3279, 1144, 1688, 1103, 3467, 5820...",Pile-CC,"[('Senate', 'Government Organization'), ('Fede..."
2,2,"This weekend during the TCA event, it was anno...","[101, 1188, 5138, 1219, 1103, 157, 11356, 1856...",Pile-CC,"[('Arrow', 'TV Show'), ('Flash', 'TV Show'), (..."
3,3,A Jeweler's Eye for Flaw\n\nStories\n\nChristi...,"[101, 138, 23624, 1200, 112, 188, 9329, 1111, ...",Pile-CC,"[('Christie Hodgen', 'Person'), ('Frederick Ba..."
4,4,SHARE\n\nMcDonald’s Family Classic turns out t...,"[101, 17730, 12426, 2036, 9092, 787, 188, 3921...",Pile-CC,"[('McDonald’s Family Classic', 'event'), ('Sch..."


In [ ]:
datapath_ent = '627entities.csv'
df_ent = pd.read_csv(datapath_ent, engine='python', on_bad_lines='skip')

df_ent.head
df_ent.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'original_text', 'tokenized_text',
       'pile_set_name', 'generated_entities'],
      dtype='object')

In [ ]:
import pandas as pd
import ast
from collections import Counter

# Replace 'your_file.csv' with the path to your CSV file
datapath_ent = '627entities.csv'
df_ent = pd.read_csv(datapath_ent, engine='python', on_bad_lines='skip')

# Initialize a Counter object to count unique entities
entity_counter = Counter()

# Define a set to hold unique entity types
entity_types = set()

# Regular expression to match entities in the form of ('Entity', 'Type')
entity_pattern = re.compile(r"\('([^']*)', '([^']*)'\)")

# Loop through each row in the 'generated_entities' column
for row in df_ent['generated_entities']:
    # Check if the row is a string
    if isinstance(row, str):
        # Find all matches of the entity pattern in the row
        entities = entity_pattern.findall(row)

        # Update the Counter with entities
        entity_counter.update([entity[0] for entity in entities])

        # Add entity types to the set
        entity_types.update([entity[1] for entity in entities])

# Print the number of unique entities
print(f"Number of unique entities: {len(entity_counter)}")
# Print the number of unique entity types
print(f"Number of unique entities: {len(entity_types)}")
# Print the unique entity types
print(f"Unique entity types: {entity_types}")

# Optional: Save the unique entities and their counts to a CSV
pd.DataFrame(entity_counter.items(), columns=['Entity', 'Count']).to_csv('unique_entities.csv', index=False)

# Optional: Save the unique entity types to a text file
with open('unique_entity_types.txt', 'w') as f:
    for entity_type in entity_types:
        f.write(f"{entity_type}\n")

Number of unique entities: 7592
Number of unique entities: 521
Unique entity types: {'application', 'Dataset', 'Software', 'Certification', 'device', 'weather', 'DJ', 'gender', 'Record Label', 'FS', 'position', 'work_of_art', 'artist', 'Ethnicity', 'Concept Art', 'Court', 'brand', 'material', 'Band', 'nationality', 'health condition', 'Musical Group', 'Vehicle', 'extension', 'Medical Condition', 'Collection', 'service', 'Ingredient', 'Unit', 'amount of money', 'Album', 'award', 'Music Band', 'object', 'chance', 'Animal', 'Music Brand', 'Activity', 'Sports Venue', 'Work of Art', 'department', 'creative work', 'Text', 'Work', 'User', 'Other', 'State', 'Clothing', 'process', 'template designer', 'time', 'Anatomical Structure', 'Organization', 'Lens Type', 'League', 'hex color', 'season', 'location', 'example', 'Grant', 'outcome', 'Component', 'chemical compound', 'Event', 'technology', 'Product Line', 'Sports Team', 'Educational Institution', 'operating system', 'custom demo', 'PRODUCT', 